In [102]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output

In [73]:
mbb_url = ["https://docs.google.com/spreadsheets/d/1BcIP7CIYDTNnedcRG3U3HAIwluaJtz8LCAKfArlEh98/edit#gid=2032963821".replace('/edit#gid=', '/export?format=csv&gid='),
            "https://docs.google.com/spreadsheets/d/1BcIP7CIYDTNnedcRG3U3HAIwluaJtz8LCAKfArlEh98/edit#gid=1904120862".replace('/edit#gid=', '/export?format=csv&gid='),
            "https://docs.google.com/spreadsheets/d/1BcIP7CIYDTNnedcRG3U3HAIwluaJtz8LCAKfArlEh98/edit#gid=2056191088".replace('/edit#gid=', '/export?format=csv&gid=')]

wbb_url = ["https://docs.google.com/spreadsheets/d/12JWqAMfVrSZobLohmxQK6PyrbNbOQbM6ux4LxXqWenM/edit#gid=2032963821".replace('/edit#gid=', '/export?format=csv&gid='),
           "https://docs.google.com/spreadsheets/d/12JWqAMfVrSZobLohmxQK6PyrbNbOQbM6ux4LxXqWenM/edit#gid=748348799".replace('/edit#gid=', '/export?format=csv&gid='),
           "https://docs.google.com/spreadsheets/d/12JWqAMfVrSZobLohmxQK6PyrbNbOQbM6ux4LxXqWenM/edit#gid=2056191088".replace('/edit#gid=', '/export?format=csv&gid=')]

In [74]:
mbb_game_dates = ["10/28/22", "11/1/22"]
wbb_game_dates = ["10/29/22", "11/5/22"]
teams_played = ['Hope', 'Elmhurst', 'Concordia', 'Edgewood']

## Data

In [77]:
regions = {'P' : 'Paint', 
           'LC' : 'Left Corner', 
           'LE' : 'Left Elbow', 
           'RE' : 'Right Elbow',
           'TK' : 'Top of Key', 
           'LW' : 'Left Wing', 
           'RW' : 'Right Wing', 
           'T' : 'Top', 
           'RB' : 'Right Baseline', 
           'RC' : 'Right Corner',
           'LB' : 'Left Baseline'}

In [78]:
post_regions = {'L' : 'Left',
               'C' : 'Center',
               'R' : 'Right'}

post_distances = {'S' : 'Short',
                  'L' : 'Long'}

In [79]:
shot_types = {'S' : 'Catch & Shoot (Set)',
              'M' : 'Catch & Shoot (Moving)',
              'F' : 'Floater',
              'H' : 'Hook Shot',
              'L' : 'Layup',
              'B' : 'Stepback',
              'P' : 'Pull-Up'}

In [80]:
turnover_types = {'DRIVE' : 'Drive',
                  'DP' : 'Drive + Pass',
                  'PASS' : 'Reversal/Pass',
                  'PER' : 'Perimeter/Strip',
                  'POST' : 'Post Entry',
                  'SCREEN' : 'Off-Ball',
                  'KEY' : 'Off-Ball'}

## Datasets

In [93]:
def load_data(team):
    if team:
        current = mbb_url
        current_dates = mbb_game_dates
    else:
        current = wbb_url
        current_dates = wbb_game_dates

    shots_df = pd.read_csv(current[0])
    posts_df = pd.read_csv(current[1])
    tovs_df = pd.read_csv(current[2])

    regions3 = ['RC', 'LC', 'LW', 'RW', 'T']
    shots_df['Range'] = shots_df.Region.apply(lambda i : i in regions3)
    shots_df['Range'] = shots_df['Range'].map({False : "2-Point", True : "3-Point"})

    shots_df['Game'] = shots_df.Date.apply(lambda i : i in current_dates)
    posts_df['Game'] = posts_df.Date.apply(lambda i : i in current_dates)
    tovs_df['Game'] = tovs_df.Date.apply(lambda i : i in current_dates)

    shots_df = shots_df[(~shots_df.Game)]
    posts_df = posts_df[(~posts_df.Game)]
    tovs_df = tovs_df[(~tovs_df.Game)]
    
    shots_df.Region = shots_df.Region.map(regions)
    shots_df['Shot Type'] = shots_df['Shot Type'].map(shot_types)

    posts_df.Region = posts_df.Region.map(post_regions)
    posts_df.Distance = posts_df.Distance.map(post_distances)
    
    return [shots_df, posts_df, tovs_df]

In [94]:
mbb = load_data(True)
wbb = load_data(False)

## Player Report

In [168]:
def pick_team(Team):
    global selected_team
    global players
    
    selected_team = Team
        
    if Team == "Men's Team":
        players = ['All Players'] + list(pd.unique(mbb[0].Player))
    else:
        players = ['All Players'] + list(pd.unique(wbb[0].Player))
    
    clear_output()
    interact(player_report, Player=players)
    
    return None

In [169]:
def player_report(Player):
    if selected_team == "Men's Team":
        shots_df = mbb[0]
        posts_df = mbb[1]
        tovs_df = mbb[2]
    else:
        shots_df = wbb[0]
        posts_df = wbb[1]
        tovs_df = wbb[2]
    
    if Player != 'All Players':
        shots_df = shots_df[shots_df.Player == Player]
        posts_df = posts_df[posts_df.Player == Player]
        tovs_df = tovs_df[tovs_df.Player == Player]
    
    range_table = shots_df.groupby('Range')['Outcome'].agg([np.sum, len])
    range_table.columns = ['FGM', 'FGA']
    range_table.loc['Total'] = list(range_table.sum().values)
    range_table['FG%'] = 100 * range_table['FGM'] / range_table['FGA']
    range_table = range_table.sort_values(by='FG%', ascending=False).round(1)
    
    region = shots_df.groupby('Region')['Outcome'].agg([np.sum, len])
    region.columns = ['FGM', 'FGA']
    region['FG%'] = 100 * region['FGM'] / region['FGA']
    region = region.sort_values(by='FG%', ascending=False).round(1)

    shot_type = shots_df.groupby('Shot Type')['Outcome'].agg([np.sum, len])
    shot_type.columns = ['FGM', 'FGA']
    shot_type['FG%'] = 100 * shot_type['FGM'] / shot_type['FGA']
    shot_type = shot_type.sort_values(by='FG%', ascending=False).round(1)
    
    post_region = posts_df.groupby('Region').count()
    post_region = pd.DataFrame( post_region.iloc[:, 1] )
    post_region.columns = ['Count']
    post_region = post_region.sort_values(by='Count', ascending=False)
    
    post_distance = posts_df.groupby('Distance').count()
    post_distance = pd.DataFrame( post_distance.iloc[:, 1] )
    post_distance.columns = ['Count']
    post_distance = post_distance.sort_values(by='Count', ascending=False)

    turnover_type = tovs_df.groupby('Turnover Type').count()
    turnover_type = pd.DataFrame( turnover_type.iloc[:, 1] )
    turnover_type.columns = ['Count']
    turnover_type = turnover_type.sort_values(by='Count', ascending=False)

    clear_output()
    display(range_table)
    display(region)
    display(shot_type)
    if len(post_region) != 0:
        display(post_region)
    if len(post_distance != 0):
        display(post_distance)
    display(turnover_type)

In [170]:
interact(pick_team, Team=["Men's Team", "Women's Team"])

interactive(children=(Dropdown(description='Team', options=("Men's Team", "Women's Team"), value="Men's Team")…

<function __main__.pick_team(Team)>